In [1]:
import csv
import struct
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import expit

from skimage import measure
from skimage import filters
from skimage import exposure
from sklearn.linear_model import LogisticRegression

x = np.loadtxt("train_x.csv", delimiter=",")

In [2]:
#Image segmentation
def canny_edges(x):

    dim = int(np.sqrt(x.shape[0]))

    temp = exposure.adjust_gamma(x, (np.mean(x) ** 2) / 90 / 40)
    temp = temp.reshape(-1, dim, dim)
    temp = filters.gaussian(temp, 0.5)
    temp = (254 * np.divide(temp, np.max(temp))).astype(int)

    x = exposure.adjust_gamma(x, 10)
    x = x.reshape(-1, dim, dim)
    x = filters.gaussian(x, 0.5)
    x = (254 * np.divide(x, np.max(x))).astype(int)

    contours = measure.find_contours(temp[0], np.mean(temp) + (np.std(temp)), fully_connected='high')

    value = 225

    while not (len(contours) >= 3):
        contours = measure.find_contours(temp[0], value, fully_connected='high')
        value -= 1

        if value == 50:
            break

    contours.sort(key=len)
    contours = contours[-3:]

    point = []
    newimages = []

    if len(contours) == 3:
        for i in range(0, 3):
            point.append((np.max(contours[i], axis=0) + np.min(contours[i], axis=0)) / 2)

            x_coordinate = int(point[i][0])
            if x_coordinate < 14:
                x_coordinate = 14
            elif x_coordinate > 50:
                x_coordinate = 50
            else:
                x_coordinate = int(point[i][0])

            y_coordinate = int(point[i][1])
            if y_coordinate < 14:
                y_coordinate = 14
            elif y_coordinate > 50:
                y_coordinate = 50
            else:
                y_coordinate = int(point[i][1])

            newimages.append(x[0][x_coordinate - 14:x_coordinate + 14, y_coordinate - 14:y_coordinate + 14])

    else:
        newimages = np.zeros((3, 28, 28))

    return newimages

#Image segmentation
newimages = []
for i in range(len(x)):
    newimages.append(canny_edges(x[i]))
    newimages[i] = np.array(newimages[i])
    newimages[i] = newimages[i].reshape(-1, 28 * 28)

In [3]:
#Rewriting x to be combined vectors of the segmented images
x=np.empty([len(newimages),len(newimages[0][0])*3])

for i in range(0,len(x)):
    for k in range(0,3):
        for j in range(0,len(newimages[0][0])):
            x[i][j+k*len(newimages[0][0])]=newimages[i][k][j]

In [4]:
#Load y data
y = np.loadtxt("train_y.csv", delimiter=",")

samp=len(x)
y = y.reshape(-1, 1) 
inputsize=len(x[0])

classes=([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20, 21, 24, 25, 27, 28, 30, 32, 35, 36, 40, 42, 45, 48, 49, 54, 56, 63, 64, 72, 81])
classno=len(classes)

In [5]:
#Converting x to binary
for i in range (0,len(x)):
    for j in range (0,len(x[i])):
        x[i][j]=round((x[i][j])/255,0)   
       
 #Converting y to one-hot. (0000100000-format)
yhot=np.empty([samp,classno])
for i in range(0,len(y)):
    for j in range(0,classno):
        if y[i]==classes[j]:
            yhot[i][j]=1
        else:
            yhot[i][j]=0
            
yoriginal=y            
y=yhot

In [6]:
#To be set by cross-validation:
nodes=30
hidlays=2
traininghorizon=10000000
learningrate=1.0

np.random.seed(2)

#random extract 1/5 of x and y:
indices = np.random.permutation(x.shape[0])
training_id, test_id = indices[:(samp*4)/5], indices[(samp*4)/5:]
trainingx, testx = x[training_id,:], x[test_id,:]
trainingy= y[training_id,:]
testyoriginal=yoriginal[test_id,:]
trainyoriginal=yoriginal[training_id,:]

trainsamp=len(trainingx)
testsamp=len(testx)

#Cross validation loop:
for crossval in range(1,2):
    #hidlays=crossval
    #nodes=crossval*10
    #learningrate=0.00001*np.power(10,crossval)
    #traininghorizon=np.power(10,crossval)

    #Initialize weights to random numbers between -1 and 1
    weights=[]
    weights.append(2*np.random.random((inputsize,nodes)) - 1)
    for i in range(0,hidlays-1):
        weights.append(2*np.random.random((nodes,nodes)) - 1)
    weights.append(2*np.random.random((nodes,classno)) - 1)

    for j in xrange(traininghorizon):
        #Reducing training rate half way through:
        if j==traininghorizon/2:
            learningrate=learningrate/10
        
        #Pick a training example
        ex=np.random.randint(0,trainsamp)
        outputlay=hidlays+1

        #Feed forward example through network
        layers=[]
        layers.append(trainingx[ex])
        for i in range(0,outputlay):
            layers.append(expit(np.dot(layers[i],weights[i])))

        errors=[None] * (outputlay+1)

        #Compute correction of output
        errors[outputlay]=layers[outputlay]*(1-layers[outputlay])*(trainingy[ex]-layers[outputlay])

        # Computing share of error for all nodes in each former layer
        for i in range(hidlays, 0, -1):
            errors[i]=layers[i]*(1-layers[i])*(errors[i+1].dot(weights[i].T))

        #Change weights
        for i in range(hidlays,-1,-1):
            weights[i] +=((layers[i][np.newaxis]).T.dot(errors[i+1][np.newaxis]))*learningrate

    testprediction=[]
    for testrow in range (0,testsamp):
        testestimate=expit(testx[testrow].dot(weights[0]))
        for j in range(0,hidlays):
            testestimate=expit(testestimate.dot(weights[j+1]))
        testprediction.append(np.argmax(testestimate))

    testcorrect=0.0
    testfalse=0.0
    testcorrectpred=[]
    testfalsepred=[]

    for i in range(0,testsamp):
        if testprediction[i]==testyoriginal[i][0]:
            testcorrect+=1
            testcorrectpred.append(testprediction[i])
        else:
            testfalse+=1
            testfalsepred.append(testprediction[i])

    testaccuracy=testcorrect/(testfalse+testcorrect)
    
    trainprediction=[]
    for testrow in range (0,trainsamp):
        trainestimate=expit(trainingx[testrow].dot(weights[0]))
        for j in range(0,hidlays):
            trainestimate=expit(trainestimate.dot(weights[j+1]))
        trainprediction.append(np.argmax(trainestimate))

    traincorrect=0.0
    trainfalse=0.0
    traincorrectpred=[]
    trainfalsepred=[]

    for i in range(0,trainsamp):
        if trainprediction[i]==trainyoriginal[i][0]:
            traincorrect+=1
            traincorrectpred.append(trainprediction[i])
        else:
            trainfalse+=1
            trainfalsepred.append(trainprediction[i])

    trainaccuracy=traincorrect/(trainfalse+traincorrect)
    
    #print("Hidden layers: "+str(hidlays)+"  Test Accuracy: "+str(testaccuracy)+"  Training Accuracy: "+str(trainaccuracy))
    #print("Nodes: "+str(nodes)+"  Test Accuracy: "+str(testaccuracy)+"  Training Accuracy: "+str(trainaccuracy))
    #print("Learning rate: "+str(learningrate)+"  Test Accuracy: "+str(testaccuracy)+"  Training Accuracy: "+str(trainaccuracy))
    print("Epochs: "+str(traininghorizon)+"  Test Accuracy: "+str(testaccuracy)+"  Training Accuracy: "+str(trainaccuracy))

Epochs: 100000  Test Accuracy: 0.1475  Training Accuracy: 0.161625


In [7]:
#Pre-processing of test data set
xkaggle = np.loadtxt("test_x.csv", delimiter=",")

newimages = []
for i in range(len(xkaggle)):
    newimages.append(canny_edges(xkaggle[i]))
    newimages[i] = np.array(newimages[i])
    newimages[i] = newimages[i].reshape(-1, 28 * 28)
    
xkaggle=np.empty([len(newimages),len(newimages[0][0])*3])

for i in range(0,len(xkaggle)):
    for k in range(0,3):
        for j in range(0,len(newimages[0][0])):
            xkaggle[i][j+k*len(newimages[0][0])]=newimages[i][k][j]

kagglesamp=len(xkaggle)
y = y.reshape(-1, 1) 
inputsize=len(xkaggle[0])

#Converting x to binary
for i in range (0,len(xkaggle)):
    for j in range (0,len(xkaggle[i])):
        xkaggle[i][j]=round((xkaggle[i][j])/255,0)  



In [8]:
#Predicting the test set:
kaggleprediction=[]
for testrow in range (0,kagglesamp):
    kaggleestimate=expit(xkaggle[testrow].dot(weights[0]))
    for j in range(0,hidlays):
        kaggleestimate=expit(kaggleestimate.dot(weights[j+1]))
    kaggleprediction.append(np.argmax(kaggleestimate))

In [9]:
#Exporting the result
with open("result.csv", "wb") as file:
	wr = csv.writer(file)
	for i in range(0, len(kaggleprediction)):
		wr.writerow((i+1,kaggleprediction[i]))